### IMPORTS

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
# auxiliary funtionsto import
from utils import score, display, syncing

### LOAD DATA

In [2]:
# Read in data files
DATA_DIR = 'data/nfl-health-and-safety-helmet-assignment'

In [3]:
# Labels and sample submission
labels = pd.read_csv(f'{DATA_DIR}/train_labels.csv')
ss = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

# Player tracking data
tr_tracking = pd.read_csv(f'{DATA_DIR}/train_player_tracking.csv')
te_tracking = pd.read_csv(f'{DATA_DIR}/test_player_tracking.csv')

# Baseline helmet detection labels
tr_helmets = pd.read_csv(f'{DATA_DIR}/train_baseline_helmets.csv')
te_helmets = pd.read_csv(f'{DATA_DIR}/test_baseline_helmets.csv')

# Extra image labels
img_labels = pd.read_csv(f'{DATA_DIR}/image_labels.csv')

In [4]:
SUBMISSION_COLUMNS = ss.columns # Expected submission columns
print(f'SUBMISSION_COLUMNS: {SUBMISSION_COLUMNS.values}')

SUBMISSION_COLUMNS: ['video_frame' 'label' 'left' 'width' 'top' 'height']


### SCORE

In [5]:
# The sample submission meets these requirements.
score.check_submission(ss)

True

In [6]:
compute_naive_scores = False

if compute_naive_scores:

    # Initialize a helper scorer to to assist in scoring the local predictions.
    # Note that this is not the exact code used in the kaggle scoring system, but results should be nearly identical.
    scorer = score.NFLAssignmentScorer(labels)

    # Score the sample submission
    ss_score = scorer.score(ss)
    print(f'Sample submission scores: {ss_score:0.4f}')

    # Score a submission with only impacts
    perfect_impacts = labels.query('isDefinitiveImpact == True and isSidelinePlayer == False')
    imp_score = scorer.score(perfect_impacts)
    print(f'A submission with perfect predictions only for impacts scores: {imp_score:0.4f}')

    # Score a submission with only non-impacts
    perfect_nonimpacts = labels.query('isDefinitiveImpact == False and isSidelinePlayer == False')
    nonimp_score = scorer.score(perfect_nonimpacts)
    print(f'A submission with perfect predictions only for non-impacts scores: {nonimp_score:0.4f}')

    # Score a perfect submission
    perfect_train = labels.query('isSidelinePlayer == False')[SUBMISSION_COLUMNS].copy()
    perfect_score = scorer.score(perfect_train)
    print(f'A perfrect training submission scores: {perfect_score:0.4f}')

After scoring, the sub_labels dataframe within the NFLAssignmentScorer object can be used to evaluate results including the iou between predictions and ground truth boxes and isCorrect for correct labels. Ground truth fields have the suffix _gt while submission fields have the suffix _sub.

In [11]:
if compute_naive_scores:
    scorer.sub_labels.head()

### DISPLAY

In [12]:
create_labeled_video = False
if create_labeled_video:
    example_video = f'{DATA_DIR}/train/57584_000336_Sideline.mp4'
    output_video_path = display.video_with_baseline_boxes(example_video, tr_helmets, labels)
else:
    output_video_path = 'labeled_57584_000336_Sideline.mp4'

display.display_video(output_video_path)

### SYNCING

In [17]:
# syncing the 2D data to the video frames
tr_tracking = syncing.add_track_features(tr_tracking)
te_tracking = syncing.add_track_features(te_tracking)